# Proyecto: Naive Bayes con Estimación KDE para Mantenimiento Predictivo

---

## Objetivo

Diseñar e implementar un clasificador propio basado en **Naive Bayes**, en el que la verosimilitud $P(x_i \mid y)$ se estime utilizando técnicas de **Kernel Density Estimation (KDE)** en lugar de asumir una distribución normal.

Se utilizará el **AI4I 2020 Predictive Maintenance Dataset**, con la variable objetivo `Machine failure`. El proyecto buscará responder empíricamente si reemplazar la suposición gaussiana clásica por KDE resulta en una mejora real en desempeño.

---

## Fundamento teórico

### Naive Bayes

El clasificador **Naive Bayes** es un modelo probabilístico basado en el **teorema de Bayes** con una fuerte asunción de independencia entre las variables dado el valor de la clase. Su objetivo es calcular la probabilidad posterior de una clase $y$ dado un vector de atributos $\mathbf{x} = (x_1, x_2, \dots, x_n)$:

$$
P(y \mid \mathbf{x}) = \frac{P(y) \cdot P(\mathbf{x} \mid y)}{P(\mathbf{x})}
$$

Dado que $P(\mathbf{x})$ es constante para todas las clases en un problema de clasificación, la predicción se basa en:

$$
\hat{y} = \arg\max_y \; P(y) \prod_{i=1}^{n} P(x_i \mid y)
$$

La clave está en estimar adecuadamente las distribuciones de las verosimilitudes $P(x_i \mid y)$.


### Gaussian Naive Bayes

En el enfoque **Gaussian Naive Bayes (GNB)**, se hace la **asunción paramétrica** de que cada variable numérica sigue una distribución **normal univariada** dentro de cada clase. Es decir:

$$
P(x_i \mid y) = \mathcal{N}(x_i \mid \mu_{iy}, \sigma^2_{iy}) = \frac{1}{\sqrt{2\pi \sigma^2_{iy}}} \exp\left( -\frac{(x_i - \mu_{iy})^2}{2\sigma^2_{iy}} \right)
$$

Donde $\mu_{iy}$ y $\sigma^2_{iy}$ se estiman directamente desde los datos de entrenamiento.

Este supuesto simplifica el modelo, pero puede ser problemático si los datos no siguen una distribución gaussiana, por ejemplo, si son **multimodales, sesgados o tienen colas pesadas**.


### KDE como alternativa para estimar la verosimilitud

Una alternativa interesante a asumir una distribución normal es usar un enfoque **no paramétrico** para estimar la densidad de probabilidad: la **Kernel Density Estimation (KDE)**.

En este método, la verosimilitud $P(x_i \mid y)$ se estima directamente a partir de los datos de entrenamiento, sumando pequeñas funciones kernel centradas en cada observación. La forma general es:

$$
\hat{f}(x) = \frac{1}{n h} \sum_{i=1}^{n} K\left( \frac{x - x_i}{h} \right)
$$

Donde:
- $K$ es la función kernel (por ejemplo, gaussiana, triangular, rectangular),
- $h$ es el parámetro de suavizado o **bandwidth**,
- $x_i$ son las observaciones de la clase $y$.

Este enfoque permite capturar formas de distribución **arbitrarias** y potencialmente más realistas que una gaussiana simple, lo cual podría mejorar la clasificación en contextos donde la suposición de normalidad no se cumple.

En este proyecto, exploraremos esta idea usando tres variantes de KDE y comparándolas contra Gaussian Naive Bayes, evaluando cuál ofrece un mejor desempeño en el contexto de mantenimiento predictivo.


---

## Dataset

- **Nombre:** AI4I 2020 Predictive Maintenance Dataset  
- **Fuente:** UCI / Kaggle  
- **Target:** `Machine failure` (binaria)  
- **Características:** Datos de sensores, condiciones operativas, tiempos de uso  
- **Consideración crítica:** Dataset **desbalanceado**, con una minoría de fallas reales.  
- Se recomienda realizar una revisión de la literatura sobre cómo se ha abordado el desbalance en este dataset.

---

## Metodología

### 1. Implementación de clasificador Naive Bayes personalizado

- Se implementará un clasificador Naive Bayes desde cero.
- Para cada clase $y$, se estima la verosimilitud de cada variable continua $x_i$ como $P(x_i \mid y)$ usando métodos no paramétricos de KDE.
- Se comparará el rendimiento del modelo con un `GaussianNB` de `sklearn` como baseline.



### 2. Evaluación de tres métodos de KDE

Cada grupo deberá comparar los siguientes tres métodos de estimación de densidad para calcular la verosimilitud $P(x_i \mid y)$.



#### a. KDE clásico con kernel gaussiano + optimización de `bandwidth`

- Se utiliza un kernel gaussiano para estimar la densidad:
  
  $$
  \hat{f}(x) = \frac{1}{n h \sqrt{2\pi}} \sum_{i=1}^{n} \exp\left( -\frac{(x - x_i)^2}{2h^2} \right)
  $$

- El parámetro `bandwidth` $h$ controla la suavidad de la estimación.
- Para seleccionar un valor apropiado de $h$, se debe usar un método de optimización como **Grid Search** o preferiblemente **optimización bayesiana**, evaluando el rendimiento del modelo en términos de **AUC ROC** mediante validación cruzada (al menos 5 folds).



#### b. KDE tipo Parzen (ventanas simples)

- En este enfoque, se reemplaza el kernel gaussiano por una ventana de forma simple, como rectangular o triangular.

  - **Ventana rectangular (tophat):**
    $$
    K(u) = \frac{1}{2} \cdot \mathbb{I}(|u| \leq 1)
    $$

  - **Ventana triangular:**
    $$
    K(u) = (1 - |u|) \cdot \mathbb{I}(|u| \leq 1)
    $$

- El modelo general de densidad estimada sigue siendo:

  $$
  \hat{f}(x) = \frac{1}{n h} \sum_{i=1}^{n} K\left( \frac{x - x_i}{h} \right)
  $$

- El ancho de ventana $h$ debe definirse manualmente y mantenerse fijo.
- Puede implementarse directamente o usando `sklearn.neighbors.KernelDensity` con `kernel='tophat'` o `'linear'`.


#### c. KDE con regla de Silverman

- Se utiliza un kernel gaussiano, pero el `bandwidth` se calcula automáticamente con la **regla de Silverman**:

  $$
  h = 1.06 \cdot \hat{\sigma} \cdot n^{-1/5}
  $$

- La estimación de la densidad sigue la misma forma que el KDE gaussiano clásico:

  $$
  \hat{f}(x) = \frac{1}{n h \sqrt{2\pi}} \sum_{i=1}^{n} \exp\left( -\frac{(x - x_i)^2}{2h^2} \right)
  $$

- Este método no requiere ajuste de hiperparámetros.
- Se puede implementar fácilmente con `scipy.stats.gaussian_kde`.



---

## Instrucciones de evaluación

- **Métrica principal:** Área bajo la curva ROC (AUC ROC)
- **Procedimiento de validación:** Validación cruzada estratificada de 5 folds
  - Usar el mismo esquema de partición para todos los métodos para asegurar comparabilidad.
  - Calcular el AUC promedio y su desviación estándar para cada modelo.
  - Registrar los tiempos de procesamiento

- **Modelos a comparar:**
  - Naive Bayes clásico (`GaussianNB`. Pueden usar el de Scikit Learn)
  - Naive Bayes con KDE:
    - KDE con kernel gaussiano y bandwidth ajustado por optimización
    - KDE con ventana tipo Parzen
    - KDE con regla de Silverman

- **Objetivo del análisis:**
  - Determinar si vale la pena reemplazar la suposición de normalidad por KDE en este problema.
  - Comparar las tres variantes de KDE y explicar cuál se comporta mejor y bajo qué condiciones (tamaño del bandwidth, forma de la distribución, impacto del desbalance, etc.).
  - Identificar si el ajuste del bandwidth influye significativamente en el desempeño del modelo correspondiente.
  - Comparar el tiempo de computación de cada método y analizar su impacto en la viabilidad práctica.

---

## Consideraciones adicionales

- **Desbalanceo:**
  - Investigar y aplicar estrategias adecuadas para el tratamiento del desbalance en el dataset (por ejemplo: submuestreo, sobreponderación, uso de métricas balanceadas).
  - Buscar artículos académicos (Google Scholar, Scopus, etc.) que hayan trabajado con el AI4I 2020 dataset y reportado técnicas o resultados relevantes para manejar el desbalanceo.
  
- **Preprocesamiento de variables:**
  - Analizar y justificar la eliminación o transformación de columnas irrelevantes, constantes o ruidosas.
  - Evaluar si es necesario aplicar técnicas de **normalización** o **estandarización** sobre las variables numéricas.
    - Si se aplican, explicar qué método se usó (e.g., `StandardScaler`, `MinMaxScaler`) y por qué.
    - Si no se aplican, justificar por qué no se consideran necesarias dadas las características del modelo y del dataset.

- **Visualización (opcional pero recomendada):**
  - Graficar las curvas de densidad estimadas para algunas variables por clase, con diferentes métodos de KDE.
  - Mostrar gráficamente el efecto del parámetro `bandwidth` en la forma de la distribución.

---

## Entregables

- **Código fuente documentado:** en formato Jupyter Notebook o scripts Python bien estructurados.
- **Repositorio en GitHub:**  
  - El proyecto completo debe entregarse en un repositorio público o privado (según indicaciones), incluyendo:
    - Código fuente
    - Archivo README con instrucciones de ejecución
    - Presentación en formato `.pptx` o `.pdf`
    - Cualquier recurso adicional necesario para reproducir los experimentos

- **Presentación oral (15 + 5 minutos):**
  - 15 minutos de exposición + 5 minutos de preguntas por parte del docente o compañeros.
  - Debe incluir:
    - Descripción clara de los métodos comparados
    - Justificación del preprocesamiento realizado
    - Visualizaciones de las distribuciones de verosimilitud (KDE) y curvas ROC
    - Reporte de AUC promedio, luego de una validación cruzada de al menos 5 folds, de cada modelo analizado
    - Análisis crítico: ¿usar KDE mejora la clasificación frente al modelo gaussiano tradicional? ¿bajo qué condiciones?
    - Comparación de tiempos de computación entre los métodos.

---

## Recomendaciones

- Leer artículos recientes que hayan trabajado con el **AI4I 2020 Predictive Maintenance Dataset**, en especial aquellos que discuten estrategias para manejar el **desbalance de clases**. Usar fuentes confiables como Google Scholar o Scopus.

- Apoyarse en la **teoría revisada en clase** y en el **material del curso disponible en el repositorio oficial**. Revisar los notebooks de ejemplos, apuntes, y recursos adicionales proporcionados por el docente.

- Se recomienda fuertemente el uso de herramientas de inteligencia artificial como **ChatGPT** o **GitHub Copilot** para:
  - Aclarar dudas conceptuales.
  - Obtener sugerencias de código.
  - Redactar fragmentos documentados.
  - Mejorar el entendimiento de errores o ajustes técnicos.

> El uso de estas herramientas debe ser **ético, transparente y reflexivo**. No se trata de copiar código sin entenderlo, sino de apoyarse en IA como una forma de acelerar el trabajo, validar ideas y fomentar buenas prácticas.

---

## Referencias

- Los artículos académicos utilizados como referencia deben ser mencionados al final de la presentación en formato **IEEE**.

---

## Fecha de entrega

- **Límite de entrega:** domingo **22 de julio**, hasta las **23:59 (hora local)**.
- El proyecto debe entregarse mediante un **repositorio en GitHub**, incluyendo:
  - Código funcional y documentado
  - Informe en formato `.pptx` o `.pdf`
  - Instrucciones para reproducir los resultados

---

## Rúbrica de Evaluación (100 puntos)

### Código y reporte técnico — 50 puntos

| Criterio                                                                 | Puntos |
|--------------------------------------------------------------------------|--------|
| Implementación funcional de los tres métodos de KDE                      | 15     |
| Optimización de `bandwidth` (KDE gaussiano) usando validación cruzada con AUC como métrica | 10     |
| Comparación cuantitativa con GaussianNB usando AUC en validación cruzada (5 folds) | 5      |
| Documentación clara en el notebook o scripts (comentarios, celdas explicativas, encabezados) | 10     |
| Visualización y análisis de resultados (curvas de densidad, curvas ROC, resumen de AUC) | 5      |
| Justificación y evidencia sobre el manejo del desbalance y preprocesamiento aplicado | 5      |


### Presentación oral — 50 puntos

| Criterio                                                                 | Puntos |
|--------------------------------------------------------------------------|--------|
| Explicación clara del enfoque Naive Bayes y motivación de usar KDE       | 10     |
| Comparación detallada entre métodos de KDE y GaussianNB                  | 10     |
| Justificación clara de decisiones de diseño y preprocesamiento           | 10     |
| Análisis crítico de resultados                                          | 10     |
| Claridad, manejo del tiempo, calidad visual de la presentación (diapositivas) | 10     |

---

**Total: 100 puntos**

PASO 1: CARGA DEL DATASET Y SELECCION DE VARIABLES RELEVANTES


In [1]:
import pandas as pd

# Cargar datos
df = pd.read_csv("ai4i2020.csv")
print("Shape:", df.shape)
df.head()


Shape: (10000, 14)


,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


DEFINICION Y VISTA RAPIDA DE VARIABLES IMPORTANTES


In [2]:
# Ver tipos y columnas
df.dtypes.to_frame("dtype")


,dtype
UDI,int64
Product ID,object
Type,object
Air temperature [K],float64
Process temperature [K],float64
Rotational speed [rpm],int64
Torque [Nm],float64
Tool wear [min],int64
Machine failure,int64
TWF,int64


Target (y): Machine failure (0 = no falla, 1 = falla).

Features numéricas:

    -Air temperature [K]

    -Process temperature [K]

    -Rotational speed [rpm]

    -Torque [Nm]

    -Tool wear [min]

Feature categórica:

    -Type (L, M, H) → la vamos a codificar luego.

NO usar como features (porque son IDs o causan fuga de información):

    -UDI

    -Product ID

    -TWF, HDF, PWF, OSF, RNF (son tipos específicos de falla, están derivados del target).

In [3]:
# Definir columnas que se usarán más adelante
feature_cols_num = [
    "Air temperature [K]",
    "Process temperature [K]",
    "Rotational speed [rpm]",
    "Torque [Nm]",
    "Tool wear [min]"
]

target_col = "Machine failure"

print("Features numéricas:", feature_cols_num)
print("Target:", target_col)
print("\nClases de 'Machine failure':")
print(df[target_col].value_counts())


Features numéricas: ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']
Target: Machine failure

Clases de 'Machine failure':
Machine failure
0    9661
1     339
Name: count, dtype: int64


PASO 2: CONSTRUCCION DE LA MATRIZ DE CARACTERISTICAS Y DIVISION 80/20 ESTRATIFICADA

Objetivo:

Armar la matriz de features X (numéricas + tipo de máquina).

Definir el vector objetivo y (Machine failure).

Dividir en train/test con estratificación para respetar el desbalan

--CONSTRUIR X E Y

In [4]:
from sklearn.model_selection import train_test_split
import numpy as np

# 1) Features numéricas (ya definidas)
X_num = df[feature_cols_num].copy()

# 2) Codificar la variable categórica 'Type' con one-hot encoding
X_type = pd.get_dummies(df["Type"], prefix="Type")

# 3) Unir numéricas + categóricas
X = pd.concat([X_num, X_type], axis=1)

# 4) Vector objetivo
y = df[target_col].astype(int)

print("Shape X:", X.shape)
print("Shape y:", y.shape)
X.head()


Shape X: (10000, 8)
Shape y: (10000,)


,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Type_H,Type_L,Type_M
0,298.1,308.6,1551,42.8,0,False,False,True
1,298.2,308.7,1408,46.3,3,False,True,False
2,298.1,308.5,1498,49.4,5,False,True,False
3,298.2,308.6,1433,39.5,7,False,True,False
4,298.2,308.7,1408,40.0,9,False,True,False


--TRAIN/TEST SPLIT ESTRATIFICADO

In [5]:
# División 80/20 estratificada según la variable objetivo (Machine failure)
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("X_train:", X_train.shape, "X_test:", X_test.shape)
print("y_train distribución:")
print(y_train.value_counts(normalize=True))
print("\ny_test distribución:")
print(y_test.value_counts(normalize=True))


X_train: (8000, 8) X_test: (2000, 8)
y_train distribución:
Machine failure
0    0.966125
1    0.033875
Name: proportion, dtype: float64

y_test distribución:
Machine failure
0    0.966
1    0.034
Name: proportion, dtype: float64


Paso 3: Baseline con Gaussian Naive Bayes (sklearn)

Objetivo:
Entrenar un Naive Bayes “normal” de sklearn y obtener métricas base para luego comparar contra nuestros modelos con KDE.

In [6]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 1) Crear y entrenar el modelo
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# 2) Predecir en el conjunto de test
y_pred_gnb = gnb.predict(X_test)

# 3) Métricas básicas
acc = accuracy_score(y_test, y_pred_gnb)
cm = confusion_matrix(y_test, y_pred_gnb)

print("=== Gaussian Naive Bayes (Baseline) ===")
print(f"Accuracy: {acc:.4f}")
print("\nMatriz de confusión:")
print(cm)

print("\nReporte de clasificación (clase 1 = falla):")
print(classification_report(y_test, y_pred_gnb, digits=4))


=== Gaussian Naive Bayes (Baseline) ===
Accuracy: 0.9580

Matriz de confusión:
[[1908   24]
 [  60    8]]

Reporte de clasificación (clase 1 = falla):
              precision    recall  f1-score   support

           0     0.9695    0.9876    0.9785      1932
           1     0.2500    0.1176    0.1600        68

    accuracy                         0.9580      2000
   macro avg     0.6098    0.5526    0.5692      2000
weighted avg     0.9450    0.9580    0.9506      2000

